In [1]:
import pandas as pd, tensorflow as tf,  matplotlib.pyplot as plt, sys
from sklearn.preprocessing import MinMaxScaler

from matplotlib.ticker import MultipleLocator, FormatStrFormatter, FuncFormatter
from plot_styles import set_plotstyles
set_plotstyles()
%config InlineBackend.figure_format = 'retina'

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/Users/andrew/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#import data
data_path_1 = 'data/wu_spikey.csv'
data_df_1 = pd.read_csv(data_path_1, dtype = float)

# split random 80% / 10%
training_data_df_1 = data_df_1.sample(frac = 0.8)
testing_data_df_1 = data_df_1.loc[~data_df_1.index.isin(training_data_df_1.index)]

# separate input features from output
X_training_1 = training_data_df_1.drop('slope', axis = 1).values
y_training_1 = training_data_df_1[['slope']].values

X_testing_1 = testing_data_df_1.drop('slope', axis = 1).values
y_testing_1 = testing_data_df_1[['slope']].values

# setup data scalers
X_scaler_1 = MinMaxScaler(feature_range = (0,1))
y_scaler_1 = MinMaxScaler(feature_range = (0,1))

# scale data
X_scaled_training_1 = X_scaler_1.fit_transform(X_training_1)
y_scaled_training_1 = y_scaler_1.fit_transform(y_training_1)

X_scaled_testing_1 = X_scaler_1.transform(X_testing_1)
y_scaled_testing_1 = y_scaler_1.transform(y_testing_1)

In [3]:
number_of_inputs = X_scaled_training_1.shape[1]
number_of_outputs = y_scaled_training_1.shape[1]

In [4]:
num_layers = 3
layer_1_nodes = 768
layer_2_nodes = 512
layer_3_nodes = 256

learning_rate = 0.0001

In [19]:
# change log path
run_flag = 'C_dataset_spikey_LR_0001_first_leaky_relu_long'
nodes = '_'.join(['{:03}'.format(x) for x in [layer_1_nodes, layer_2_nodes, layer_3_nodes]])
logdir = '{:02}_layers_{}_nodes_run_{}'.format(num_layers, nodes, run_flag)
logdir

'03_layers_768_512_256_nodes_run_C_dataset_spikey_LR_0001_first_leaky_relu_long'

In [20]:
tf.reset_default_graph()

In [21]:
with tf.variable_scope('input'):
    X  = tf.placeholder(tf.float32, shape = (None, number_of_inputs))
    
with tf.variable_scope('layer_1'):

    biases = tf.get_variable(name = "biases1", shape = [layer_1_nodes], initializer = tf.zeros_initializer())
    weights = tf.get_variable(name = "weights1", shape  = [number_of_inputs, layer_1_nodes], initializer = tf.contrib.layers.xavier_initializer())
    layer_1_output = tf.nn.leaky_relu(tf.matmul(X, weights) + biases)

with tf.variable_scope('layer_2'):

    biases = tf.get_variable(name = "biases2", shape = [layer_2_nodes], initializer = tf.zeros_initializer())
    weights = tf.get_variable(name = "weights2", shape  = [layer_1_nodes, layer_2_nodes], initializer = tf.contrib.layers.xavier_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

with tf.variable_scope('layer_3'):

    biases = tf.get_variable(name = "biases3", shape = [layer_3_nodes], initializer = tf.zeros_initializer())
    weights = tf.get_variable(name = "weights3", shape  = [layer_2_nodes, layer_3_nodes], initializer = tf.contrib.layers.xavier_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

    
with tf.variable_scope('output'):

    biases = tf.get_variable(name = "biases_out", shape = [number_of_outputs], initializer = tf.zeros_initializer())
    weights = tf.get_variable(name = "weights_out", shape  = [layer_3_nodes, number_of_outputs], initializer = tf.contrib.layers.xavier_initializer())
    prediction = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)

with tf.variable_scope('cost'):

    y = tf.placeholder(tf.float32, shape = (None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, y))

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate, epsilon=1e-08).minimize(cost)

with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    tf.summary.histogram('predictions', prediction)
    summary = tf.summary.merge_all()

In [22]:
saver = tf.train.Saver()

In [23]:
starting_epoch = 0
training_epochs = 200

In [24]:
with tf.Session() as session:

    session.run(tf.global_variables_initializer())

    training_writer = tf.summary.FileWriter('logs/' + logdir + '/training', session.graph)
    testing_writer = tf.summary.FileWriter('logs/' + logdir + '/testing', session.graph)

    for epoch  in range(training_epochs):
        epoch += starting_epoch
        # Feed in the training data and do one stepf NN training
        session.run(optimizer,
                    feed_dict = {X: X_scaled_training_1, y: y_scaled_training_1})

        training_feed = {X: X_scaled_training_1, y: y_scaled_training_1}
        training_cost, training_summary = session.run([cost, summary],
                                                feed_dict = training_feed)

        testing_feed =  {X: X_scaled_testing_1, y: y_scaled_testing_1}
        testing_cost, testing_summary = session.run([cost, summary],
                                                     feed_dict = testing_feed)
        sys.stdout.write('\r epoch: {:4.0f} testing_cost: {:2.20f}'.format(epoch, testing_cost))
        
        training_writer.add_summary(training_summary, epoch)
        testing_writer.add_summary(testing_summary, epoch)
        testing_writer.flush()
        training_writer.flush()

    model_save_location = 'logs/' + logdir + '/model' + '/trained_model.ckpt'
    save_path = saver.save(session, model_save_location)

 epoch: 1136 testing_cost: 0.00472165038809180260

KeyboardInterrupt: 

RuntimeError: Attempted to use a closed Session.